## 价值迭代


#### 1、创建环境

In [2]:
import numpy as np
import random
def get_state(row, col):
    if row != 3:
        return 'ground'
    if row == 3 and col == 0:
        return 'ground'
    if row == 3 and col == 11:
        return 'terminal'
    
    return 'trap'

get_state(0, 0)
    

'ground'

#### 2、动作空间

In [3]:
def move(row, col ,action):
    if get_state(row, col) in ['trap', 'terminal']:
        return row, col, 0
    
    if action == 0:
        row -= 1

    if action == 1:
        row += 1 

    if action == 2:
        col -= 1 

    if action == 3:
        col += 1

    row = max(0, row) 
    row = min(3, row) 
    col = max(0, col) 
    col = min(11, col) 
    reward = -1;
    if get_state(row, col) == 'trap':
        reward = -100
    if get_state(row, col) == 'terminal':
        reward = 0
    return row, col, reward

#### 3、Q函数


In [4]:
Q = np.zeros([4, 12 ,4])

def get_action(row, col):
    if random.random() < 0.1:
        return random.choice(range(4))
    
    return Q[row, col].argmax()

def get_update(row, col, action, reward, next_row, next_col,next_action):
    target = 0.9 *Q[next_row, next_col].max()
    target += reward

    value = Q[row, col, action]

    update = target - value  ##TD error

    update *= 0.1       ##learnning rate
    return update

get_update(0, 0 ,3, -1, 0, 1, 3)

-0.1

#### 4、训练

In [5]:
def train():
    for epoch in range(1500):
        row = random.choice(range(4))
        col = 0

        action =get_action(row, col)

        reward_sum = 0

        while get_state(row, col) not in ['terminal', 'trap']:
            next_row, next_col, reward = move(row, col , action)
            reward_sum += reward

            next_action = get_action(next_row, next_col)
            update = get_update(row, col, action, reward, next_row, next_col, next_action)
            Q[row, col,action] += update

            row = next_row
            col = next_col 
            action = next_action

        if epoch % 150 == 0:
            print(epoch, reward_sum)

train()


0 -129
150 -34
300 -103
450 -15
600 -12
750 -14
900 -110
1050 -11
1200 -12
1350 -13


##### 5、结果显示

In [6]:
#打印游戏，方便测试
def show(row, col, action):
    graph = [
        '□', '□', '□', '□', '□', '□', '□', '□', '□', '□', '□', '□', '□', '□',
        '□', '□', '□', '□', '□', '□', '□', '□', '□', '□', '□', '□', '□', '□',
        '□', '□', '□', '□', '□', '□', '□', '□', '□', '○', '○', '○', '○', '○',
        '○', '○', '○', '○', '○', '❤'
    ]

    action = {0: '↑', 1: '↓', 2: '←', 3: '→'}[action]

    graph[row * 12 + col] = action

    graph = ''.join(graph)

    for i in range(0, 4 * 12, 12):
        print(graph[i:i + 12])


show(1, 1, 0)

□□□□□□□□□□□□
□↑□□□□□□□□□□
□□□□□□□□□□□□
□○○○○○○○○○○❤


In [7]:

from IPython import display
import time
def show(row, col, action):
    graph = [
        '□', '□', '□', '□', '□', '□', '□', '□', '□', '□', '□', '□', '□', '□',
        '□', '□', '□', '□', '□', '□', '□', '□', '□', '□', '□', '□', '□', '□',
        '□', '□', '□', '□', '□', '□', '□', '□', '□', '○', '○', '○', '○', '○',
        '○', '○', '○', '○', '○', '❤'
    ]

    action = {0: '↑', 1: '↓', 2: '←', 3: '→'}[action]

    graph[row * 12 + col] = action

    graph = ''.join(graph)

    for i in range(0, 4 * 12, 12):
        print(graph[i:i + 12])


show(1, 1, 0)



def test():
    #起点
    row = random.choice(range(4))
    col = 0

    #最多玩N步
    for _ in range(200):

        #获取当前状态，如果状态是终点或者掉陷阱则终止
        if get_state(row, col) in ['trap', 'terminal']:
            break

        #选择最优动作
        action = Q[row, col].argmax()

        #打印这个动作
        display.clear_output(wait=True)
        time.sleep(0.1)
        show(row, col, action)

        #执行动作
        row, col, reward = move(row, col, action)


test()

□□□□□□□□□□□□
□□□□□□□□□□□□
□□□□□□□□□□□↓
□○○○○○○○○○○❤


In [8]:

#打印所有格子的动作倾向,策略
for row in range(4):
    line = ''
    for col in range(12):
        action = Q[row, col].argmax()
        action = {0: '↑', 1: '↓', 2: '←', 3: '→'}[action]
        line += action
    print(line)

→→→→→→→→→→↓↓
→↓→→→→↓↓→↓→↓
→→→→→→→→→→→↓
↑↑↑↑↑↑↑↑↑↑↑↑
